In [4]:
!pip install seaborn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns


In [28]:
df = pd.read_csv("data/raw/Bengaluru_House_Data.csv")
print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nData Types:")
print(df.dtypes)

print("\nTop 10 missing value columns:")


Shape: (13320, 9)

Columns:
['area_type', 'availability', 'location', 'size', 'society', 'total_sqft', 'bath', 'balcony', 'price']

Data Types:
area_type           str
availability        str
location            str
size                str
society             str
total_sqft          str
bath            float64
balcony         float64
price           float64
dtype: object

Top 10 missing value columns:


In [41]:
df

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


In [30]:
df.info()
df.isnull().sum()

<class 'pandas.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  str    
 1   availability  13320 non-null  str    
 2   location      13319 non-null  str    
 3   size          13304 non-null  str    
 4   society       7818 non-null   str    
 5   total_sqft    13320 non-null  str    
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), str(6)
memory usage: 936.7 KB


area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [48]:
df1 = df.drop('society', axis=1)
df1.shape

(13320, 8)

In [50]:
df1['BHK'] = df1['size'].str.extract(r'(\d+)').astype(float)
df1[['size', 'BHK']].head()

,size,BHK
0,2 BHK,2.0
1,4 Bedroom,4.0
2,3 BHK,3.0
3,3 BHK,3.0
4,2 BHK,2.0


In [51]:
df1 = df1.dropna(subset=['BHK'])
df1.shape

(13304, 10)

In [53]:
df2 = df1.drop('size', axis=1)
df2.head()

,area_type,availability,location,size,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07,2.0
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00,4.0
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,1440,2.0,3.0,62.00,3.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00,3.0
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,1200,2.0,1.0,51.00,2.0


In [59]:
df2['total_sqft'].unique()[:20]

<StringArray>
['1056', '2600', '1440', '1521', '1200', '1170', '2732', '3300', '1310',
 '1020', '1800', '2785', '1000', '1100', '2250', '1175', '1180', '1540',
 '2770',  '600']
Length: 20, dtype: str

In [62]:
df2[df2['total_sqft'].str.contains('-', na=False)].head()

,area_type,availability,location,size,total_sqft,bath,balcony,price,BHK
30,Super built-up Area,19-Dec,Yelahanka,4 BHK,2100 - 2850,4.0,0.0,186.000,4.0
56,Built-up Area,20-Feb,Devanahalli,4 Bedroom,3010 - 3410,NaN,NaN,192.000,4.0
81,Built-up Area,18-Oct,Hennur Road,4 Bedroom,2957 - 3450,NaN,NaN,224.500,4.0
122,Super built-up Area,18-Mar,Hebbal,4 BHK,3067 - 8156,4.0,0.0,477.000,4.0
137,Super built-up Area,19-Mar,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,0.0,54.005,2.0


In [63]:
def convert_sqft(x):
    try:
        if '-' in x:
            tokens = x.split('-')
            return (float(tokens[0]) + float(tokens[1])) / 2
        return float(x)
    except:
        return None

df2['total_sqft'] = df2['total_sqft'].apply(convert_sqft)

df2['total_sqft'].dtype

dtype('float64')

In [64]:
df2['total_sqft'].isna().sum()

np.int64(46)

In [65]:
df2 = df2.dropna(subset=['total_sqft'])
df2.shape

(13258, 9)

In [67]:
df2['location'] = df2['location'].str.strip()
df2['location'].isna().sum()

np.int64(1)

In [68]:
df2 = df2.dropna(subset=['location'])
df2.shape

(13257, 9)

In [69]:
df2['location'].nunique()

1287

In [70]:
location_stats = df2['location'].value_counts()
location_stats_less_than_10 = location_stats[location_stats <= 10]

df2['location'] = df2['location'].apply(
    lambda x: 'other' if x in location_stats_less_than_10 else x
)

df2['location'].nunique()

241

In [71]:
df2['location'].value_counts().head(10)

location
other              2873
Whitefield          538
Sarjapur  Road      397
Electronic City     304
Kanakpura Road      271
Thanisandra         236
Yelahanka           212
Uttarahalli         186
Hebbal              177
Marathahalli        175
Name: count, dtype: int64

In [72]:
df2.head()

,area_type,availability,location,size,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2.0
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4.0
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3.0
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2.0


In [75]:
df2['price_per_sqft'] = (df2['price'] * 100000) / df2['total_sqft']
df2['price_per_sqft'].describe()

count    1.325700e+04
mean     7.912825e+03
std      1.064976e+05
min      2.678298e+02
25%      4.271186e+03
50%      5.438596e+03
75%      7.313318e+03
max      1.200000e+07
Name: price_per_sqft, dtype: float64

In [76]:
df2.head()

,area_type,availability,location,size,total_sqft,bath,balcony,price,BHK,price_per_sqft
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2.0,3699.810606
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4.0,4615.384615
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3.0,4305.555556
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3.0,6245.890861
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2.0,4250.000000


In [77]:
df2['price_per_sqft'].describe()

count    1.325700e+04
mean     7.912825e+03
std      1.064976e+05
min      2.678298e+02
25%      4.271186e+03
50%      5.438596e+03
75%      7.313318e+03
max      1.200000e+07
Name: price_per_sqft, dtype: float64

In [78]:
df2 = df2[(df2['price_per_sqft'] > 1000) & (df2['price_per_sqft'] < 50000)]
df2['price_per_sqft'].describe()

count    13232.000000
mean      6652.924808
std       4100.528024
min       1166.666667
25%       4271.722691
50%       5434.782609
75%       7305.313404
max      48484.848485
Name: price_per_sqft, dtype: float64

In [80]:
df2.shape

(13232, 10)

In [81]:
df2[df2['bath'] > df2['BHK'] + 2].shape

(14, 10)

In [82]:
df2 = df2[df2['bath'] <= df2['BHK'] + 2]
df2.shape

(13161, 10)

In [83]:
df2[df2['total_sqft'] / df2['BHK'] < 300].shape

(729, 10)

In [84]:
df2 = df2[df2['total_sqft'] / df2['BHK'] >= 300]
df2.shape

(12432, 10)

In [85]:
df2.head()

,area_type,availability,location,size,total_sqft,bath,balcony,price,BHK,price_per_sqft
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2.0,3699.810606
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4.0,4615.384615
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3.0,4305.555556
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3.0,6245.890861
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2.0,4250.000000


In [86]:
df2 = df2.drop('price_per_sqft', axis=1)
df2.columns

Index(['area_type', 'availability', 'location', 'size', 'total_sqft', 'bath',
       'balcony', 'price', 'BHK'],
      dtype='str')

In [87]:
df2.head()

,area_type,availability,location,size,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2.0
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4.0
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3.0
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2.0


In [88]:
df2 = df2.drop('availability', axis=1)
df2.columns

Index(['area_type', 'location', 'size', 'total_sqft', 'bath', 'balcony',
       'price', 'BHK'],
      dtype='str')

In [89]:
df2.head()

,area_type,location,size,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2.0
1,Plot Area,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4.0
2,Built-up Area,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3.0
3,Super built-up Area,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3.0
4,Super built-up Area,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2.0
